In [ ]:
import numpy as np
import torch
import sys
import matplotlib.pyplot as plt
import cv2
import re
import PIL

## Read pfm

In [ ]:
def readPFM(file):
    file = open(file, 'rb')

    color = None
    width = None
    height = None
    scale = None
    endian = None

    header = file.readline().rstrip()
    print(header)
    print(file.readline())
    if header.decode("ascii") == 'PF':
        color = True
    elif header.decode("ascii") == 'Pf':
        color = False
    else:
        raise Exception('Not a PFM file.')

    dim_match = re.match(r'^(\d+)\s(\d+)\s$', file.readline().decode("ascii"))
    print(file.readline().decode("ascii"), dim_match)
    if dim_match:
        width, height = list(map(int, dim_match.groups()))
    else:
        raise Exception('Malformed PFM header.')

    scale = float(file.readline().decode("ascii").rstrip())
    if scale < 0: # little-endian
        endian = '<'
        scale = -scale
    else:
        endian = '>' # big-endian

    data = np.fromfile(file, endian + 'f')
    shape = (height, width, 3) if color else (height, width)

    data = np.reshape(data, shape)
    data = np.flipud(data)
    return data, scale

In [ ]:
def writePFM(file, image, scale=1):
    file = open(file, 'wb')

    color = None

    if image.dtype.name != 'float32':
        raise Exception('Image dtype must be float32.')

    image = np.flipud(image)

    if len(image.shape) == 3 and image.shape[2] == 3: # color image
        color = True
    elif len(image.shape) == 2 or len(image.shape) == 3 and image.shape[2] == 1: # greyscale
        color = False
    else:
        raise Exception('Image must have H x W x 3, H x W x 1 or H x W dimensions.')

    file.write('PF\n' if color else 'Pf\n'.encode())
    file.write('%d %d\n'.encode() % (image.shape[1], image.shape[0]))

    endian = image.dtype.byteorder

    if endian == '<' or endian == '=' and sys.byteorder == 'little':
        scale = -scale

    file.write('%f\n'.encode() % scale)

    image.tofile(file)

## Dumb way from SO


In [ ]:
depth_PNG = cv2.imread('img/test_left.png', cv2.IMREAD_UNCHANGED)
print(type(depth_PNG))

In [ ]:
plt.imshow(depth_PNG)

In [ ]:
print(depth_PNG.max(), depth_PNG.min(), depth_PNG.dtype, depth_PNG.shape)
depth_PNG = depth_PNG.astype(np.float32) / 256.0
print(depth_PNG.max(), depth_PNG.min(), depth_PNG.dtype, depth_PNG.shape)

In [ ]:
depth_PFM = cv2.imwrite('img/test_so.pfm', depth_PNG)

In [ ]:
writePFM('img/test_new.pfm', depth_PNG)

In [ ]:
img, scale = readPFM('img/test_so.pfm')
print(type(img), scale)

In [ ]:
img, scale = readPFM('img/test_new.pfm')
print(type(img), scale)

In [ ]:
plt.imshow(img)

In [ ]:
img.all() == depth_PNG.all()

## All hail freiburg group

In [ ]:
import sys
import imageio

In [ ]:
def writePFM(file, image, scale=1):
    file = open(file, 'wb')

    color = None

    if image.dtype.name != 'float32':
        raise Exception('Image dtype must be float32.')

    image = np.flipud(image)

    if len(image.shape) == 3 and image.shape[2] == 3: # color image
        color = True
    elif len(image.shape) == 2 or len(image.shape) == 3 and image.shape[2] == 1: # greyscale
        color = False
    else:
        raise Exception('Image must have H x W x 3, H x W x 1 or H x W dimensions.')

    file.write('PF\n' if color else 'Pf\n'.encode())
    file.write('%d %d\n'.encode() % (image.shape[1], image.shape[0]))

    endian = image.dtype.byteorder

    if endian == '<' or endian == '=' and sys.byteorder == 'little':
        scale = -scale

    file.write('%f\n'.encode() % scale)

    image.tofile(file)

In [ ]:
depth_png = imageio.v2.imread('img/test_left.png')
print(type(depth_PNG))
print(depth_png.dtype.name)
depth_png = depth_png.astype('float32')
print(depth_png.dtype.name)

In [ ]:
writePFM('img/freiburg_left.pfm', depth_png)

In [ ]:
img, scale = readPFM('img/freiburg_left.pfm')

In [ ]:
plt.imshow(img)